In [28]:
import os
from openai import OpenAI, AsyncOpenAI
from dotenv import load_dotenv

# access API_KEYS
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],
)

### System Content:
- Indicate the behaviour for genearating a respones based on the problem to solve
- Indicating the lower and upper threshold for each varaible


In [56]:
system_content = """
You are tasked with describing clouds in relation to different weather phenomena, such as rain, imminent rain, cyclones, and anticyclones. 
Your descriptions should be based on the following five key variables: Cloud Base, High Cloud, Low Cloud, Medium Cloud, and Total Cloud.  

Here are the guidelines for each variable:\n\n1. 
**Cloud Base**: Describe the altitude and its implication for weather conditions.\n    
    - Example: 'The cloud base is low, indicating the potential for rain.'\n\n2. 
**High Cloud**: Describe the presence and type of high-altitude clouds, indicating weather patterns.\n    
    - Example: 'High cirrus clouds are present, suggesting fair weather.'\n\n3. 
**Low Cloud**: Focus on the type and density of low-altitude clouds and their weather implications.\n    
    - Example: 'Low, dense clouds are covering the sky, indicating heavy rain.'\n\n4. 
**Medium Cloud**: Describe the mid-level clouds and what they indicate about the weather.\n    
    - Example: 'Medium-altitude cumulus clouds suggest possible light showers.'\n\n5. 
**Total Cloud**: Provide an overall assessment of cloud coverage and its impact on weather.\n    
    - Example: 'The sky is completely overcast with total cloud coverage, likely leading to continuous rain.'

### \n\nThresholds of the varaibles:\n\n. 
"Cloud Base": {
    "0-2000": "The cloud base is low, indicating the potential for rain.",
    "2001-4000": "The cloud base is at a moderate altitude, suggesting stable weather conditions.",
    "4001-6000": "The cloud base is relatively high, typically indicating fair weather.",
    "6000+": "The cloud base is very high, indicating clear and stable weather with no immediate precipitation."
  },
"High Cloud": {
    "0.0-0.25": "Few to no high clouds, indicating clear skies.",
    "0.26-0.50": "Moderate presence of high clouds, suggesting fair weather with some cloud cover.",
    "0.51-0.75": "Significant presence of high clouds, indicating possible changes in weather.",
    "0.76-1": "High cloud cover is dense, suggesting an approaching front or significant weather changes."
  },
"Low Cloud": {
    "0.0-0.25": "Few to no low clouds, indicating clear or fair weather.",
    "0.26-0.50": "Moderate presence of low clouds, suggesting possible light showers.",
    "0.51-0.75": "Significant presence of low clouds, indicating potential for rain or overcast conditions.",
    "0.76-1": "Dense low cloud cover, suggesting overcast conditions with heavy rain likely."
  },
"Medium Cloud": {
    "0.0-0.25": "Few to no medium clouds, indicating stable weather.",
    "0.26-0.50": "Moderate presence of medium clouds, suggesting potential for light showers.",
    "0.51-0.75": "Significant presence of medium clouds, indicating changes in weather conditions.",
    "0.76-1": "Dense medium cloud cover, suggesting potential for continuous rain or storms."
  },
"Total Cloud": {
    "0.0-0.25": "Clear sky with minimal cloud cover, indicating no significant weather activity.",
    "0.26-0.50": "Partly cloudy sky, suggesting fair weather with some cloud presence.",
    "0.51-0.75": "Mostly cloudy sky, indicating overcast conditions with potential for light rain.",
    "0.76-1": "Completely overcast sky, likely leading to continuous rain."
  }
    
\n\nHere are sample descriptions for specific weather phenomena:\n\n 
- **Raining**: 'Low, dense clouds dominate the sky, indicating heavy rain with a low cloud base. Medium clouds add to the overcast conditions.'\n-
- **Going to Rain**: 'The cloud base is lowering with increasing total cloud coverage. Medium clouds are thickening, suggesting imminent rain.'\n- 
- **Cyclone**: 'Total cloud coverage with a mix of high cirrus and low dense clouds, indicating the presence of a cyclone. The cloud base is significantly low.'\n- 
- **Anticyclone**: 'High cirrus clouds are scattered with minimal total cloud coverage, indicating fair weather associated with an anticyclone.'\n\n

### Weather Predictions Based on Cloud Data:\n\n1. 
**Precipitation**:\n    
    - **Rain**: Presence of low, dense, and thick clouds like nimbostratus or cumulonimbus suggests rain or thunderstorms.\n    
    - **Snow**: Similar to rain, but typically observed in colder conditions. Thick, low clouds can indicate snowfall.\n\n2. 
**Clear Weather**:\n    
    - **Fair Weather**: High, wispy clouds like cirrus indicate fair weather. Scattered cumulus clouds can also suggest clear skies.\n\n3. 
**Storms and Severe Weather**:\n    
    - **Thunderstorms**: Tall, towering cumulonimbus clouds are a strong indicator of thunderstorms, possibly with lightning, heavy rain, and hail.\n    
    - **Cyclones**: Total cloud coverage with organized patterns, including low dense clouds and high cirrus clouds, can indicate cyclonic activity.\n\n4. 
**Imminent Weather Changes**:\n    
    - **Incoming Rain**: Lowering cloud base with increasing medium and low cloud coverage can suggest imminent rain.\n   
    - **Fronts and Changes in Weather**: The presence of certain clouds like altostratus can indicate a warm front, which often precedes longer periods of precipitation.\n\n5. 
**Atmospheric Stability**:\n    
    - **Stable Atmosphere**: High, thin clouds like cirrostratus or cirrocumulus indicate a stable atmosphere, often leading to calm weather.\n    
    - **Unstable Atmosphere**: Presence of towering cumulus or cumulonimbus clouds indicates an unstable atmosphere, likely leading to storms.\n\n
    
By following these guidelines, you can effectively describe the observed clouds, infer weather conditions, and make predictions concisely.
    
Provide your descriptions in a maximum of three sentences.

**Do not mention the name of the varaibles Cloud base, High cloud, Low cloud, Medium cloud and Total cloud**
"""

### User Content:
- Indicate the promnt to generate a description for the different varaibles of the clouds in this case

In [62]:
user_content = {
                  "cloud_bae"   : 2000,
                  "cloud_high"  : 0.20,
                  "cloud_low"   : 0.5,
                  "cloud_mid"   : 0.74,
                  "cloud_total" : 0.80
                }

prompt = f"""
            The weather data is as follows:
            Cloud Base: {user_content['cloud_base']} meters
            High Cloud: {user_content['cloud_high']*100}%
            Low Cloud:  {user_content['cloud_low']*100}%
            Medium Cloud: {user_content['cloud_mid']*100}%
            Total Cloud: {user_content['cloud_total']*100}%

            Based on this data, provide an analysis of the cloud coverage and any potential implications for weather conditions.
        """

In [63]:
client = AsyncOpenAI()
completion = await client.chat.completions.create(
    model="gpt-4o", 
    messages= [
    {
        "role": "system",
        "content": system_content
    },
    
    {
        "role": "user",
        "content": prompt
    }
  ],
  temperature=0.7
)

In [64]:
print(completion.choices[0].message)

ChatCompletionMessage(content='The cloud base is at a moderate altitude, suggesting stable weather conditions. Moderate high clouds indicate mostly clear skies at higher altitudes, while significant presence of low clouds suggests the potential for rain or overcast conditions. Dense medium clouds contribute to an almost completely overcast sky, indicating imminent rain is likely.', role='assistant', function_call=None, tool_calls=None)
